# Versions of the Timeseries Quantum Autoencoder
*TS QAE in Qiskit with angle encoding of TS sliding windows*

By: Jacob Cybulski<br>
Date: August 2023 - March 2024<br>
Aims: The goal of this notebook is to build a Time Series Quantum Autoencoder, <br/>
    a circuit which can compress a quantum state of a timeseries onto a smaller amount of qubits, <br/>
    while retaining the information from the initial state.
Methods: A QAE model from Qiskit Tutorial 12 has been adopted for time series processing. 
- Time series data was converted to a series of sliding windows.
- Several approaches to TS data encoding were tested, i.e. unary, binary and anglular.
- Angle encoding was eventually used, with values centered around H state, in the range of [-pi/2..pi/2]
- The model featuring an input and encoder blocks only (followed by a swap test) was subsequently trained.
- A decoder was then added for testing, and the circuit initialised with an optimum set of parameters from training.
- Each test sample was then encoded into the full QAE circuit, which was executed using a state vector simulation.
- State vectors of input and output data was then visualised and compared. 

Sources: 
1. Romero, Jonathan, Jonathan P. Olson, and Alan Aspuru-Guzik. 2017. “Quantum Autoencoders for Efficient Compression of Quantum Data.”<br/>
   Quantum Science and Technology 2 (4): 045001.
3. Bravo-Prieto, Carlos, "Quantum autoencoders with enhanced data encoding." Machine Learning: Science and Technology, 2, May 2021
4. Qiskit Tutorial, https://qiskit.org/ecosystem/machine-learning/tutorials/12_quantum_autoencoder.html. *Based on [1].*
5. Eugenia Anello, Denoising Autoencoder in Pytorch on MNIST dataset, Dataseries (Medium), Jun 28, 2021.
6. Eugenia Anello, <a href="https://github.com/eugeniaring/Medium-Articles/blob/main/Pytorch/denAE.ipynb">GitHub</a>, Jun 28, 2021.
7. Phillip Lippe, Tutorial 9: Deep Autoencoders, UvA Deep Learning Tutorials, Fall 2022.

Notes:
- Refer to the end of the notebook for the log of changes
- Algorithm by Romera was followed
- Results of the enhanced algorithm by Bravo-Prieto could not be reproduced

## Initial settings

In [1]:
%%html
<style>
table {float:left}
</style>

## References

1. A wikipedia page on Autoencoder: https://en.wikipedia.org/wiki/Autoencoder

2. Romero, Jonathan, Jonathan P. Olson, and Alan Aspuru-Guzik. "Quantum autoencoders for efficient compression of quantum data." Quantum Science and Technology 2.4 (2017): 045001.

3. Swap Test Algorithm: https://en.wikipedia.org/wiki/Swap_test

4. Bravo-Prieto, Carlos, "Quantum autoencoders with enhanced data encoding." Machine Learning: Science and Technology, 2, May 2021

## Versions

**Qiskit QAE with Angle Encoding**
- V1 Using QAE for anomaly detection (by denoising)
    - V1.1 Tested binary encoding, which misses proximity
    - V2.0 Created an "unary" encoding scheme
    - V2.1 Unary encoding tested with various options, not close
      - Issues noted:
          - Issue 01 (Completed) *Not sure how to interpret the output - cannot be visual*
          - Issue 02 (Completed) *Visual match between input and output not close*
          - Issue 03 (Completed) *Noise needs to be injected in training*
          - Issue 04 (Completed) *Training is getting slow, need to use GPU*
          - Issue 05 (Completed) *It is worth testing if an increase of the latent or trash layers could improve the result*
      - Action planned:
          - *Add Aer GPU*, *Add noise in training*, *Add convolution*, *Create classical solution to compare*
    - V2.2 Adding GPU plus encoding test
      - Encoding test:
          - Tested amplitude encoding. To improve the QAE we may need to focus on other aspects of the model,<br/>
            e.g. incorporation of noise and improvement to the ansatz to ensure neighbourhoods are strongly
            entangled.
      - Several problems installing GPU support, attempt to address Issue 04
          - Issue 06 (Completed) *Several problems associated with GPU support*<br/>
          - **Fix 06** *Need to install ML 0.7.1+ and Aer 0.13.2+*<br/>
            ML 0.7.1 standard installation (no need to install from source anymore)<br/>
            Aer 0.13.2 requires CUDA + cuQuantum + cuTensor (Deb)<br/>
            - Note that Aer Sampler has a bug and will not work with RawFeatureVector.
          - **Fix 04** *With GPU the training speed increased ~2 times
    - V2.3 Attempt to move to PuTorch, abandoned for the time being
    - V2.4 Changing to angle encoding - relative with Ry over 2pi
      - Issues noted:
          - Issue 07 (Completed) *Negative numbers not coded properly*</br>
      - Action planned:
          - *Change encoding as relative from H over pi* - with [-1:0] pi/2 negatives towards |0> and [0:+1] pi/2 positives towards |1>
    - V2.5 Completed QAE with angle encoding, lots of fixes
      - Fixes completed:
          - **Fix 07** *Adopted encoding around H state, up negative, down positive*
          - **Fix 02** *Resulting QAE outputs better match the inputs*
          - **Fix 01** *With the new encoding, the output is easy to interpret*
      - Issues noted:
          - *Issue 08* (Partially Completed) *QAE reconstruction has large variance end of windows*, averaging overlapping windows may help
          - Issue 09 (Completed) *Need to reconstruct the entire series*, possibly with averaging of overlapping windows
      - Action planned:
          - Issue 10 (Completed) *Systematically experiment with:* data sets, window size, step size, trash size, qubit no, ansatz reps and entanglement type
    - V2.6 Testing various QAE configurations
      - Varied the number of qubits (8 was best) and asatz reps (4 was best)
      - Issues noted:
          - Issue 11 (Completed) *Random seed seems to have little effect on results*
          - Issue 12 (Completed) *QAE results tend to be close to zero at the window ends, e.g. 1st and 7th value [0..7]*
          - Issue 13 (Completed) *Related to #12, the first window value should not be zero but the diff with the value from the previous window*<br/>
            Change the order of processing - first create deltas, then split into windows, change TS value encoding and decoding (no need to delta)
    - V2.7 Testing with an older computer Tomahawk, 64Gb RAM, 2 x GeForce GTX 1080 Ti, also testing the enhance encoding of Bravo-Prieto [4]
      - Issues noted:
          - Issue 14 (Completed) *GPU failed with error:*<br/>
            Simulation failed and returned the following error message:<br/>
            ERROR:  [Experiment 0] This program was not compiled for SM 61<br/>
          - Issue 15 (Completed) *A series of experiments with Bravo-Prieto's algorithm failed to produce any results*
      - Fixes completed:
          - **Fix 14** Reason (as provided on Slack), CuQuantum does not support GTX 1080 GPUs (cat.SM 61) - may not be fixable, GPU too old
          - **Fix 15** Bravo-Prieto algorithm could not be reproduced in any form, and with all possible options - it was abandoned*
      - CPU run works well - faster than Lenovo, results are pretty good (as per V2.6)
    - V2.8 Fixed incorrect coding of the first window value
      - Fixes completed:
          - **Fix 11** *Added **seed-simulator** to the backend options for GPU, this may be added for CPU when using AerSampler for CPU*
          - **Fix 12-13** *The first window encoded value is now delta of the first value minus the last value from the previous window*<br/>
            This was achieved by encoding the entire time series first, before splitting it into windows<br/>
            Additional charts were provided to show the original data, encoded sequence, and testing sequence<br/>
            QAE results at the edges of a window are non zero
      - Issues notes:
          - Issue 16 (Completed) *Noted high accuracy of results in the middle of windows, however, the edges are distorted* - related to Issue 08 and issue 12.<br/>
            The causes of this phenomenon may be: (1) the ansatz entanglement strategy, (2) position of Swap Test and recovery of the trash space, or (3) the sequential presentation of input.<br/>
            A possible approach would be to shuffle data on input, move the position of the trash to the middle, or to reject the values on edges.<bt/>
            To test this we need to experiments with larger windows
    - V2.9 Tested QAE with windows of size 10 - great results
      - Fixes completed:
          - Fix 08 (Completed) *When testing with windows of size 8 and 10, the edge inaccuracy persists but shifts*<br/>
            This may be the feature of the approach with Swap Test training
      - Action planned:
        This run was saved and the generated parameters and data will be used to address **Issue 9** (reconstruction of the series)
    - V2.10 Started a cleanup of all code
      - A test was conducted to create a QAE with 5/6 latent and 3 trash qubits. Its results were disappointing. The notebook, however, was copied to the Runs folder.
    - V2.11 Finished cleanup
    - V2.12 Adding precision stats of QAE results - R2, RMS, MAE and MAPE, with optional edge trimming of windows to increase precision
    - V2.13 Extendend the cost function, now part of the class "Cost"<br/>
      - Work conducted:
          - Implemented class "Cost", which allows parameterisation of the function behaviour.
          - Attempt 16 (Fully experimented) with the following results:
            - *Implemented shuffling of data durinug training, but there was no impact on the edge distortion*
            - *Tested variety of entanglement options, "sco" was found to be the best, other strategies produced distortions elsewhere and inferior results, so this may still be the culprit*
            - *Position of the trash space (and the Swap Test) seems to coincide with the edge distortion, so it could be its cause - needs a separate experiment*
    - V2.14 Merging all QAE test windows into a single time series (considering windows step and edge trimming)
      - Fixes completed:
        - **Fix 05** Experimented with different size of trash / latent area were conducted. with different outcomes, as reported.
        - **Fix 08, 16** The edge distortion is possibly linked to the choice of the ansatz, which moves the majority of variance to the edges.
          However, elimination of qubits causing distortion may in fact act contrary tothe very idea of QAE restoring the entire range of values in
          both latens and trash area. So this must be done with caution.
        - **Fix 09** The full series can now be reconstructured, with averaging of overlapping windows, and taking into consideration of window trimming.
        - **Fix 10** The systematic tests of the QAE has commensed and will continue.
    - V2.15 Test Summary, includes comments on Test 1, Test 2, ..., Test 6, and the Summary notebook
      - Observations:
        - The previous tests (such as v2.14_merged_ts), focused primarily on perfecting QAE where the trainnig and test sets were drawn from the same data sample.
          It seemed that the recovered input patterns had some edge distortion, i.e. increase in variance near qubit 0 and max, removal of which greatly improved the QAE accuracy.
          What was not obvious at the time was the fact that the distortion would most commonly occur in the trash area.
          This also means that by trimming the trash-related window components carried the majority of noise, and with a small trash area, the latent area was trained to approximate an identity.
        - In Test 1, the training sample and a test data were drawn from different sample types, however, both were of an oscillating nature, but of a different complexity and amplitude.
          The initial intuition indicated that the recovered data would match the patterns occuring in training data. However, all data recovered would match that of a test data sample -
          this is what QAE is in fact supposed to do.
        - Test 2a was further testing the hypothesis that QAE would recover the patterns present in a test sample. However, when the types of training and test samples are significantly different, i.e.
          sinusuidal and polynomila (almost linear differences betweel data points), the QAE had difficuties replicating the polynomial input. This could possibly be attributed to the QAE input coding
          which consisted of qubit rotations, and which favour Fourier style of data processing rather than that typical of regression or polynomial data analysis.
          The Test 2b seems to indicate that the QAE is not suitable for learning linear relationships, inspite of excellent cost achieved at the end - note that Test 2c excludes overtrainnig.
          It is also worth noting the errors generated by simply measuring the near-linear windows data, their R2 is as low (negative!) as that of the recovered signal!
        - Test 3 further confirmed the QAE ability to correctly recover test data, as long as the training data matched its type. However, having used the random oscillations in training
          data, it was a great surprise to see the QAE abillity to correctly recover test data. The hypothesis at this point of time was that QAE, without any intelligence, simply replicates i
          qubits spanning the latent area.
        - Test 4 again looked at polynomial data with near-linear test set. The results show very poor alignment (in terms of R2) between the orginal (but differenced) data and its recovered input.
          As before in Test 2, it showed that the error generated by QAE for input recovered on output is of the same magnitude as that generated by the circuit consisting of encoded input alone.
          The variance of measured input and the output values around the expected windows values look dramatic, however, this is illusory as the default y range in the chart is very small.
          It is worth noting that when looking at the results in the scale of the original data, their MAE and RMS are relatively small.
        - Tests 5, 6 and 7 aimed to verify whether or not the QAE was able to abstract the patters learnt from the training data and transfer this knowledge to test data.
          Data used for these experiments involved the USA beer sales over several weeks. In the tests, the trash area was progressively enlarged from 2 qubits (leaving 6 qubits in a latent area),
          to 3 (with 5 latent qubits) and then 4 (with 4 latent qubits). It was evident that the QAE was able to approximately recover the input, however, the errors would enlarge (as evidenced by
          R2, RMSE, MAE and MAPE measurement).
        - Test 8 was conducted to ensure that the latent space did not simply produce a copy of the input, which was found not to be the case.
        - <font color="green">Test 9 was to replace swap test with in-line test and a new cost function cost=1-P(|0>^t) where t is the size of trash. The approach resulted in faster execution but lower accuracy.</font>
        - In summary, the QAE works well. It is able to abstract training data, to allow recovery of previously unseen input data on its output. It is the richness of training data set, which determines
          the accuracy of QAE processing with test data, rather than the similarity of the two data sets. However, a completely different types of samples used in training and testing (such as
          oscillating vs linear) will prevent QAE to perform with any degree of accuracy.
    - V2.16 Development of the full-QAE which could then be used to support noise elimination, and allow the full input vs output comparison
      - Issues noted:
          - **Issue 17** Training of the full-QAE is ***50 times slower*** than of the half-QAE.
            The reasons for this is probably due to the Swap Test on 8 qubits rather than on just the trash space.
            Another likely reason is that the Swap Test is comparing arbitrary states rather than |0> states.
            Note that a Swap Test may not be effective when the states are close to |+>, which is our encoding of zero
            (see Siegelwax, B.N., 2020. The Simplest Way to Compare Single-Qubit Quantum States. Medium).
            A more efficient swap test may need to be used. Two possible solutions are considered at this stage:<br/>
            (1) train the model in two stages (first as half-QAE for noisy data, then as full-QAE for noisy vs clean data);<br/>
            (2) use a destructive swap on one qubit for state swapping and one ancilla for collection and measurement;<br/>
            (3) somehow reduce comparison of arbitrary states to zero states.e.g. by rotating all qubits back and then checking for |0>.
    - V2.17 Full-QAE with de-noising
      - Tests completed:
          - Value scaling was tested, large interval generates incorrect TS
          - Using full-QAE as a simple encoder S(x) A(x) S(x) with Swap Test to zero works, however, results are not as good as with half-QAE.
          - Three tests conducted to test the full-QAE performance with varying sizes of latent space, i.e. 2.17a (6+2), 2.17b (5+3), 2.17c (4+4)
      - Issues noted:
          - Issue 18 (Completed) Noise embedded in a circuit is not edequate as it is fixed for the entire duration of training.
            Noise needs to vary for all instances of training data instead.
          - Issue 19 (Completed) Scaling of pure and noisy signals seems to be incorrect, noise seems to have reduced amplitude of the signal.
            At the moment noise is added to the original signal. However, after differencing it seems to be distorted - this will affect the work of QAE in denoising.
            Perhaps noise needs to be added after differencing, so that it is not subtracted or amplified?
          - Issue 20 (Completed) At the moment the full-QAE incorrectly implements a decoder as an encoder-dag, it should be a separate block with its own parameters,
            which unfortunately will double execution time and slow it even more.
          - Issue 21 (Completed) Full-QAE does not learn to produce clean data from noisy data, it seems to pass noise from input to output.
            This may be related to the fact that optimisation error is higher than the discrepancy between noisy and pure data.
          - Issue 22 (Completed) Replace the expensive QAE cost function, with a simple P(|0>^n), which can be calculated as the count of all zeros / number of shots.
            Idea - aim is to maximise zeros, so measure all qubits, in cost function collect result of forward, check result[0] (all zeros), max this!
      - Fixes completed:
          - **Fix 03, 18** Now the noise is injected into the TS itself for training or testing
          - **Fix 19** Scaling to interval (-1, 1) results in incorrect interpretation of measurements (due to encoding), should not be attempted.
            Scaling with (-0.5, +0.5) works better.
    - V2.18 Full-QAE with de-noising and new cost function cost=1-P(|0>^q) (q=the number of qubits)
      - Observations:
          - The new full-QAE is twice as fast
          - It takes half the number of qubits
          - It is marginally less accurate
          - Does not address issue 21 (and possibly 20)
      - Fixes completed:
          - **Fix 22** The new cost function cost=1-P(|0>^q) (q=the number of qubits) has been implemented and tested.
    - V2.19 Full-QAE has a separate encoder and decoder
      - Observations:
          - The new full-QAE with separate encoder/decoder performs slightly better than encoder/encoder-dg
          - It is still unable to reproduce good TS from bad ones (issue 21), perhaps it needs on input a mix of good and noisy data.
      - Fixes completed:
          - **Fix 20** Full-QAE encoder is followed by an independent decoder-dg, same structure as encoder but in reverse order with different parameters.
    - V2.20 Implemented a new cost function which counts the number of 1s in measurements, aiming to eliminate them
      - Observations:
          - The optimisation very rapidly converges to lower values
          - However, it takes lots of iterations to achieve minimum
          - However, the minimum does not seem to be what is needed
          - The reconstructed curve is not changing much
    - V2.21 Going back to cost function 1-P(|0>^n), drastic reduction in data samples used for QAE training and validation
      - Observations:
          - Data sample was reduced to 29 examples for training and 10 for validation
          - The model seems to train quicker and smoother but requires more iterations (500)
          - The model actually generalises away from its training data and learns pure data from noise
          - Performance metrics are reasonable but not great
          - Much better results than before
      - Fixes completed:
          - **Fix 21** The model learns to reconstruct pure data windows from noisy data
      - Future work:
          - Test the model performance on other (simpler) data sets
          - Test the model performance with the cost function minimising the 1s in measurements
          - Try pretraining the encoder with half-QAE and stage decoder training (this may improve the training speed and address the only outstanding **Issue 17**)
    - V3.00 Conducted several tests of model training with different sizes of latent/trash sizes, results still need to be interpreted
    - V3.01 The cost class was extended to allow model MAE testing during training, additional plots were also added at the bottom to chart the collected performance info.
    - V3.03 Testing with more samples for training and validation
      - Issues notes:
        - **Issue 23** Modify to allow cost+parameters to be saved at some intervals only (with specific step number).
        - **Issue 24** Data analysis ignores the windows step and assumes it to be 1.
        - **Issue 25** All analysis should move to a separate notebook where the best model could be rebuilt and used.
      - Issues fixed:
        - **Fix 23** Intervals for logging and shuffling added
        - **Fix 24** Windows step has been addressed in the analysis but has only been added to "xx_v3_03_testing_beer_500_lat5_test.ipynb"
    - V3.04 Split into components: data prep, training, analysis and plots
      - Added LOG/CASE structure
      - Multi-noise recordings
      - Multiple recordings within LOG database + default info.json for the last case (training + analysis)
      - Adding PyTorch classical model test (excellent)
    - V3.05 Experiments to improve QAE
      - Issues noted:
        - **Issue 26** It seems that the number of shots has a huge impact om the results from state vector backend, e.g.<br/>
            ts_qae_v3_05_training_qiskit_lat7_xdf_1000.ipynb:<br/>
            shots=1:      CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 2000 epochs): 0.12795 min cost, 340.50 sec<br/>
            shots=10:     CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 1000 epochs): 0.33898 min cost, 280.18 sec<br/>
            shots=100:    CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 1000 epochs): 0.87636 min cost, 243.16 sec<br/>
            shots=1000:   CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 2000 epochs): 0.05051 min cost, 601.48 sec<br/>
            shots=1000:   CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 2000 epochs): 0.05184 min cost, 592.76 sec<br/>
            shots=10000:  CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 2000 epochs): 0.11094 min cost, 399.21 sec<br/>
            shots=10000:  CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 2000 epochs): 0.57737 min cost, 248.35 sec<br/>
            shots=100000: CPU Fit (58 samples, 2 reps, "linear" ent, 7+1 qubits, 96 params, 2000 epochs): 0.05430 min cost, 596.68 sec
      - Improving trainig outcomes (QAE seems to be learning input noise rather than pure output)
        - Increase the level of abstraction by reducing the latent space (failed)
        - Longer training for various options (failed)
        - Increase ansatz complexity / ruichness by adding extra layers 2, 3, 4 (failed)
        - Changing the quantum loss function from zeros to swap (failed)
        - Changing optimisation function from COBYLA to ADAM or SPSA (failed)
        - Utilising larger qubit state space by using Rx and Ry operators (failed)
        - Adding a degree of freedom by including an extra width consisting of not measured qubit (**current**)
        - Relying on a 100% classical loss function by comparing inputs with outputs directly
      - Improving training / analysis speed
        - Avoid measuring individual qubits by measuring all qubits and calculating individual qubit states -<br/>
          this improved the speed 20 times for CPU execution, which now is also much faster than GPU execution
    - V3.06 Continuation with the ability to add the circuit width
      - Conducted ICCS'2024 Qiskit tests training + analysis
        - ts_qae: All lat=7 for aw=0..5, rep=1..4 variants
        - lt_qae: All aw=3, rep=1 for lat=1..8 variants
        - la_qae: All aw=3, rep=2 for lat=1..8 variants
      - Conducted ICCS'2024 PyTorch tests training + analysis (trans from V3.04)
        - ts_qae: All lat=1, 3, 5, 7
    - V4.00 Modified the cost function Cost_Flex to accepts arbitrary objective function via parameters
      - New requirement to pass in separate data sets for QAE input and output (incompatible with legacy)
      - New parameter to pass in an objective function, which accepts:
        - Expected output (array of arrays of values to be generated by QAE)<br/>
          Note: these values correspond to angular state of qubits on output; and,
        - An array of probability distributionsresulting from quantum measurement of all qubits<br/>
          Note: that the probabilities can be used directly or to calculate angular state of each qubit on measurement.
    - V4.01 Tested the Cost_Flex with a number of objective functions (MAE, MSE, RMSE, -R2, 1-R2)
      - Updated the PyTorch versions of training and analysis notebooks
      - Changed the notebook naming convention, for Qiskit to start with "ts_qiskit_..." and for PyTorch to start with "ts_torch_..."
      - Generated a new "beer" data set of 200 flat samples, window 8 step 2, resulting 146 windows for training and 24 windows for validation
      - Results of the Qiskit Cost_Flex+ObjFuncs are poor for the old and new data sets as the training produces in a heavily averaged (flat) reconstructions
      - Results of the PyTorch QAE models for the old and new data sets are excellent
    - V4.02 Update to Qiskit V1.02
      - Moved in updated utils/* from QTSA project
      - Generated new log: beer_qiskit_100
      - Generated new beer data
    - V4.03 Migrated to PennyLane + PyTorch

# Software in use

In [1]:
!pip list | grep -E 'qiskit|torch'

qiskit                        0.45.2
qiskit-aer-gpu                0.13.2
qiskit-algorithms             0.2.2
qiskit-dynamics               0.4.3
qiskit-finance                0.4.0
qiskit-ibm-provider           0.6.3
qiskit-ibm-runtime            0.11.3
qiskit-ibmq-provider          0.20.2
qiskit-machine-learning       0.7.1
qiskit-nature                 0.7.1
qiskit-optimization           0.6.0
qiskit-qasm3-import           0.4.1
qiskit-sphinx-theme           1.12.1
qiskit-terra                  0.45.2
torch                         2.1.0
torchaudio                    2.1.0
torchsummary                  1.5.1
torchvision                   0.16.0
torchviz                      0.0.2
